In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

In [28]:
import geopandas as gpd
import pandas as pd
from siuba import *
import numpy as np

from segment_speed_utils import helpers, gtfs_schedule_wrangling
from shared_utils import rt_dates, gtfs_utils_v2, catalog_utils
import folium
import itertools
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
gcsgp = GCSGeoPandas()

In [3]:
from update_vars import (analysis_date, AM_PEAK, PM_PEAK, EXPORT_PATH, GCS_FILE_PATH, PROJECT_CRS,
SEGMENT_BUFFER_METERS, AM_PEAK, PM_PEAK, HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD, SHARED_STOP_THRESHOLD)

In [4]:
import create_aggregate_stop_frequencies as casf

In [5]:
df = pd.read_parquet(f"{GCS_FILE_PATH}max_arrivals_by_stop_single_route.parquet")

In [6]:
df

,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count
0,0089bd1b0a2b78a8590d8749737d7146,40090,6,[17_1],6.0,2.00,1.50,12.0,1
1,0089bd1b0a2b78a8590d8749737d7146,40090,1,[61_0],2.0,0.33,0.50,3.0,1
2,0089bd1b0a2b78a8590d8749737d7146,40091,4,[17_0],6.0,1.33,1.50,10.0,1
3,0089bd1b0a2b78a8590d8749737d7146,40091,1,[61_1],2.0,0.33,0.50,3.0,1
4,0089bd1b0a2b78a8590d8749737d7146,40092,4,[17_0],6.0,1.33,1.50,10.0,1
...,...,...,...,...,...,...,...,...,...
101987,f8e4fa18131802bf978177326377241d,883129,1,[605_0],NaN,0.33,0.00,1.0,1
101988,f8e4fa18131802bf978177326377241d,883130,4,[14_1],6.0,1.33,1.50,10.0,1
101989,f8e4fa18131802bf978177326377241d,883131,4,[4_0],6.0,1.33,1.50,10.0,1
101990,f8e4fa18131802bf978177326377241d,883131,1,[502_0],NaN,0.33,0.00,1.0,1


In [47]:
st, trips = casf.get_st_trips(analysis_date) # includes lookback
st_prepped = casf.add_route_dir(trips=trips, stop_times=st, analysis_date=analysis_date).pipe(casf.prep_stop_times)

{'2025-08-20': ['eTrans Schedule', 'Roseville Transit GMV Schedule'], '2025-09-24': ['San Juan Capistrano Trolley Schedule', 'Culver City Schedule'], '2025-10-15': ['Yolobus Schedule', 'Go West Schedule', 'Bay Area 511 Angel Island-Tiburon Ferry Schedule', 'El Monte Schedule', 'Nevada County Schedule']}


/home/jovyan/data-analyses/high_quality_transit_areas/create_aggregate_stop_frequencies.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stop_times['peak'] = stop_times['arrival_hour'].map(peaks_dict)


In [8]:
max_arrivals_by_stop_single = st_prepped.pipe(casf.stop_times_aggregation_max_by_stop, analysis_date, single_route_dir=True)
max_arrivals_by_stop_single.to_parquet(f"{GCS_FILE_PATH}max_arrivals_by_stop_single_route.parquet") #  for branching_derived_intersections.py
max_arrivals_by_stop_multi = st_prepped.pipe(casf.stop_times_aggregation_max_by_stop, analysis_date, single_route_dir=False)

multi_only_explode = casf.get_explode_multiroute_only(max_arrivals_by_stop_single, max_arrivals_by_stop_multi, (HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD))
share_counts = {}
multi_only_explode.groupby(['schedule_gtfs_dataset_key', 'stop_id']).apply(casf.accumulate_share_count, share_counts=share_counts)
qualify_dict = {key: share_counts[key] for key in share_counts.keys() if share_counts[key] >= SHARED_STOP_THRESHOLD}

In [9]:
multi_only_explode.query('route_dir.str.contains("42")')

,schedule_gtfs_dataset_key,stop_id,route_dir
228,076e30b080fdc5501151bd3fb0a37b9e,0013,042_1
248,076e30b080fdc5501151bd3fb0a37b9e,1997,042_0
265,076e30b080fdc5501151bd3fb0a37b9e,3125,042_1
267,076e30b080fdc5501151bd3fb0a37b9e,3183,042_1
269,076e30b080fdc5501151bd3fb0a37b9e,3185,042_0
...,...,...,...
21938,d4a07b520153bd5c27860835bc42610e,840,242-13188_1
22108,d4a07b520153bd5c27860835bc42610e,9590,242-13188_0
22109,d4a07b520153bd5c27860835bc42610e,9590,242-13188_1
22480,e5f10f33e8659f958697e55d35651255,2558638,14208_0


In [10]:
stops = helpers.import_scheduled_stops(analysis_date)

In [11]:
stops = stops.merge(trips[['feed_key', 'schedule_gtfs_dataset_key']].drop_duplicates(), on='feed_key')

In [12]:
trips.query('name.str.contains("Yolo")')

,name,feed_key,schedule_gtfs_dataset_key,trip_id,route_id,direction_id,route_type,lookback_date,analysis_name


In [13]:
stops.query('schedule_gtfs_dataset_key == "076e30b080fdc5501151bd3fb0a37b9e"')

,feed_key,service_date,feed_timezone,first_stop_arrival_datetime_pacific,last_stop_departure_datetime_pacific,stop_id,stop_key,stop_name,stop_event_count,route_type_0,...,route_type_3,route_type_4,route_type_5,route_type_6,route_type_7,route_type_11,route_type_12,missing_route_type,geometry,schedule_gtfs_dataset_key
35705,cbae8712910fbb4c24027c69d481cc9e,2025-11-05,America/Los_Angeles,2025-11-05 08:19:00,2025-11-05 18:19:00,0002,580ec94e2dde98250912fdee6565c926,Del Monte Center / Gate 1,19,NaN,...,19.0,NaN,NaN,None,None,None,None,None,POINT (-169535.057 -157452.978),076e30b080fdc5501151bd3fb0a37b9e
35706,cbae8712910fbb4c24027c69d481cc9e,2025-11-05,America/Los_Angeles,2025-11-05 08:03:00,2025-11-05 19:03:00,0003,404d56bbba8b8bedeb4fad2c082e01c6,Del Monte Center / Gate 2,21,NaN,...,21.0,NaN,NaN,None,None,None,None,None,POINT (-169522.345 -157443.006),076e30b080fdc5501151bd3fb0a37b9e
35707,cbae8712910fbb4c24027c69d481cc9e,2025-11-05,America/Los_Angeles,2025-11-05 08:02:00,2025-11-05 19:02:00,0004,711f0012d88f85ce3e8bf26aff00d4b7,Del Monte Center / Gate 3,40,NaN,...,40.0,NaN,NaN,None,None,None,None,None,POINT (-169674.843 -157470.966),076e30b080fdc5501151bd3fb0a37b9e
35708,cbae8712910fbb4c24027c69d481cc9e,2025-11-05,America/Los_Angeles,2025-11-05 05:54:00,2025-11-05 21:54:00,0006,1ef80078e2f610f1218d86a1c43afd90,6th Ave / Mission St,73,NaN,...,73.0,NaN,NaN,None,None,None,None,None,POINT (-171687.260 -160634.964),076e30b080fdc5501151bd3fb0a37b9e
35709,cbae8712910fbb4c24027c69d481cc9e,2025-11-05,America/Los_Angeles,2025-11-05 05:27:00,2025-11-05 21:51:00,0011,ef0a41233926aca8cfeb763c469b2187,Northridge Mall,163,NaN,...,163.0,NaN,NaN,None,None,None,None,None,POINT (-147884.242 -143324.784),076e30b080fdc5501151bd3fb0a37b9e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36664,cbae8712910fbb4c24027c69d481cc9e,2025-11-05,America/Los_Angeles,2025-11-05 06:05:00,2025-11-05 20:10:00,9209,7c4caedabc4c2459289915de7c13e219,Salinas Transit Center / Gate 9,36,NaN,...,36.0,NaN,NaN,None,None,None,None,None,POINT (-147847.789 -147671.571),076e30b080fdc5501151bd3fb0a37b9e
36665,cbae8712910fbb4c24027c69d481cc9e,2025-11-05,America/Los_Angeles,2025-11-05 07:27:00,2025-11-05 17:55:00,9301,a6965f63fe1714dfb711b343ed32d7d9,Marina Transit Exchange Gate 1,12,NaN,...,12.0,NaN,NaN,None,None,None,None,None,POINT (-160095.188 -146568.934),076e30b080fdc5501151bd3fb0a37b9e
36666,cbae8712910fbb4c24027c69d481cc9e,2025-11-05,America/Los_Angeles,2025-11-05 05:12:00,2025-11-05 21:45:00,9302,8bada6f9a293546dbb4c50a04a892ab4,Marina Transit Exchange Gate 2,31,NaN,...,31.0,NaN,NaN,None,None,None,None,None,POINT (-160106.261 -146588.067),076e30b080fdc5501151bd3fb0a37b9e
36667,cbae8712910fbb4c24027c69d481cc9e,2025-11-05,America/Los_Angeles,2025-11-05 06:05:00,2025-11-05 21:34:00,9303,36b231b35b2cafae056e43cc9635da07,Marina Transit Exchange Gate 3,30,NaN,...,30.0,NaN,NaN,None,None,None,None,None,POINT (-160117.309 -146605.866),076e30b080fdc5501151bd3fb0a37b9e


## debug lookback

In [21]:
import lookback_wrappers
import _utils

In [15]:
ops = lookback_wrappers.read_published_operators(analysis_date)

In [16]:
ops

{'2025-08-20': ['eTrans Schedule', 'Roseville Transit GMV Schedule'],
 '2025-09-24': ['San Juan Capistrano Trolley Schedule',
  'Culver City Schedule'],
 '2025-10-15': ['Yolobus Schedule',
  'Go West Schedule',
  'Bay Area 511 Angel Island-Tiburon Ferry Schedule',
  'El Monte Schedule',
  'Nevada County Schedule']}

In [17]:
trips_cols = ["name", "feed_key", "gtfs_dataset_key", "trip_id",
           "route_id", "direction_id", "route_type"]

use ix to get crosswalk on lookback dates to ensure join..., concat after xwalk join

In [19]:
lookback_trips = lookback_wrappers.get_lookback_trips(ops, trips_cols)

In [41]:
lookback_trips_ix = lookback_wrappers.lookback_trips_ix(lookback_trips)


In [42]:
lookback_trips_ix

,name,feed_key,schedule_gtfs_dataset_key,lookback_date
0,eTrans Schedule,55c36aaf0f4ea1d12704cd1ec110be56,ea65e81b31025ca3e74e8ffb27e1a223,2025-08-20
6,Roseville Transit GMV Schedule,ccea4295c3fe5b04d4fedb0827d45487,bef2e7553d6d7fb3789f3b081e66365a,2025-08-20
0,San Juan Capistrano Trolley Schedule,9a184ffeb823895f4696fd7e1e19ff1c,ec84f97aa17ceb17faaa16f883dc2e3f,2025-09-24
1,Culver City Schedule,d2b4d780b50e242a4f10f716c5490e3a,7c7a8d65e435c2ad8aee3f0e26f9da8d,2025-09-24
0,Yolobus Schedule,6769ba94f648570ace873d1a4d17ba82,75ae96721f84cf99b890bdb8fd4099f5,2025-10-15
96,Go West Schedule,78ba934464f426a8ad74a4ecfd21c0a6,6cac58e19f149133a282fa07f9705d6d,2025-10-15
129,Bay Area 511 Angel Island-Tiburon Ferry Schedule,e331a4d3f6a3ac0c84118e64bebd2285,2005f1dd5717b537e7952677b71b6f9c,2025-10-15
273,El Monte Schedule,1f6bdbd2bd5e2e25c07750c659df6c0b,80223cf505820f3a5ee0e49c1be3e9b0,2025-10-15
340,Nevada County Schedule,9597abcbfc03d792a0ab5e0b043e42b2,eca94265831d5c499800dd921ecf4011,2025-10-15


In [ ]:
trips

In [43]:
trips = _utils.append_analysis_name(trips, analysis_date=analysis_date)

In [45]:
trips.columns

Index(['name', 'feed_key', 'schedule_gtfs_dataset_key', 'trip_id', 'route_id',
       'direction_id', 'route_type', 'lookback_date', 'analysis_name',
       'analysis_name'],
      dtype='object')

In [31]:
gtfs_dict = catalog_utils.get_catalog('gtfs_analytics_data')

In [36]:
path = f"{gtfs_dict['schedule_tables']['dir']}{gtfs_dict['schedule_tables']['gtfs_key_crosswalk']}_{analysis_date}.parquet"

In [37]:
path

'gs://calitp-analytics-data/data-analyses/gtfs_schedule/crosswalk/gtfs_key_organization_2025-11-05.parquet'

In [39]:
df = pd.read_parquet(path)

In [40]:
df

,schedule_gtfs_dataset_key,name,schedule_source_record_id,base64_url,organization_source_record_id,organization_name,caltrans_district,counties_served,hq_city,hq_county,...,service_area_sq_miles,population,service_area_pop,subrecipient_type,primary_uza_code,reporter_type,organization_type,voms_pt,voms_do,year
0,372a06b593e1716d1c911b1d1d35bedd,Santa Ynez Mecatran Schedule,recuWhPXfxMatv6rL,aHR0cDovL2FwcC5tZWNhdHJhbi5jb20vdXJiL3dzL2ZlZW...,reckp33bhAuZlmO1M,City of Solvang,05 - San Luis Obispo / Santa Barbara,None,Solvang,None,...,<NA>,0.0,<NA>,Rural General Public Transit,NaN,Rural Reporter,"City, County or Local Government Unit or Depar...",5,<NA>,2024
1,f376bbdbbc9eb991df0b76ad83cdf686,SLO Peak Transit Schedule,rec0EeeizKvsEDfRQ,aHR0cDovL2RhdGEucGVha3RyYW5zaXQuY29tL3N0YXRpY2...,recMM99msxjmc6PPv,City of San Luis Obispo,05 - San Luis Obispo / Santa Barbara,San Luis Obispo,San Luis Obispo,San Luis Obispo,...,28,56904.0,47063,None,79147.0,Full Reporter,"City, County or Local Government Unit or Depar...",15,<NA>,2023
2,7da331ea09155d04a921748d5cfc534a,TCRTA Schedule,recTGHiEo7XvyY4EG,aHR0cDovL2RhdGEucGVha3RyYW5zaXQuY29tL3N0YXRpY2...,rec4YWVqVwJ9XqDbq,Tulare County Regional Transit Agency,06 - Fresno / Bakersfield,Tulare,Visalia,Tulare,...,4839,70628.0,473117,None,88840.0,Full Reporter,Independent Public Agency or Authority of Tran...,17,<NA>,2022
3,a129e7ea52355a94a69a42d9bbe1909a,GET Schedule,rec0As8vS0QExzIWW,aHR0cDovL2V0YS5nZXRidXMub3JnL3J0dC9wdWJsaWMvdX...,recIh3vq8jwuuJlvL,Golden Empire Transit District,06 - Fresno / Bakersfield,Kern,Bakersfield,Kern,...,111,570235.0,500977,None,4681.0,Full Reporter,Independent Public Agency or Authority of Tran...,<NA>,91,2022
4,535a5f615a37a8d9e911b893935a1446,Flixbus Schedule,recl3IyQX2prWsBVj,aHR0cDovL2d0ZnMuZ2lzLmZsaXgudGVjaC9ndGZzX2dlbm...,recEm0P5zmKExpc7p,FlixBus,07 - Los Angeles / Ventura,None,None,None,...,<NA>,NaN,<NA>,None,NaN,None,None,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,89ca51fca62fcddf2152ffd5140ca7c2,SLORTA Schedule,recHC7bwsgTPqLjX4,aHR0cHM6Ly9zbG8uY29ubmV4aW9uei5uZXQvcnR0L3B1Ym...,reciakGBN1DP9dK9N,San Luis Obispo Regional Transit Authority,05 - San Luis Obispo / Santa Barbara,San Luis Obispo,San Luis Obispo,San Luis Obispo,...,380,56904.0,278348,None,79147.0,Full Reporter,Independent Public Agency or Authority of Tran...,<NA>,41,2023
211,0ca0f71cd12243939de10cad1a5fbf9c,Santa Maria Schedule,recxPy2JOcDFGDo31,aHR0cHM6Ly9zbXJ0LnRyaXBzaG90LmNvbS92MS9ndGZzLn...,rec9zGMJgNnes75K1,City of Santa Maria,05 - San Luis Obispo / Santa Barbara,Santa Barbara,Santa Maria,Santa Barbara,...,34,143609.0,150201,None,79417.0,Full Reporter,"City, County or Local Government Unit or Depar...",25,<NA>,2024
212,b879b060b4e08c5ca3eb72ea9f69269c,StanRTA Schedule,recOpFlZv8Sc2baWH,aHR0cHM6Ly9zdGFucnRhLnJpZGVyYWxlcnRzLmNvbS9Jbm...,recSiaaMmBXW7fUZS,Stanislaus Regional Transit Authority,10 - Stockton,Stanislaus,Modesto,Stanislaus,...,1515,357301.0,550660,None,58006.0,Full Reporter,Independent Public Agency or Authority of Tran...,102,<NA>,2023
213,ff87c8304a909c02ef5045cc802864ab,Metrolink Schedule,recR28oQlTW8GMJue,aHR0cHM6Ly93d3cubWV0cm9saW5rdHJhaW5zLmNvbS9nbG...,recQp6uQAUFcenmQ6,San Bernardino County Transportation Authority,08 - San Bernardino / Riverside,San Bernardino,San Bernardino,San Bernardino,...,2953,2276703.0,2187665,None,75340.0,Full Reporter,"MPO, COG or Other Planning Agency",57,<NA>,2022


In [ ]:
df.query('')